In [31]:
import pandas as pd

# Data conversion attempt (bad output)
# Read the CSV file containing the data
df = pd.read_csv("clean_ea_2016.csv")

# Convert numeric data to appropriate data types
# Dont need state and year
numeric_columns = df.columns[2:] 
df[numeric_columns] = df[numeric_columns].replace(',', '', regex=True).astype(float)

# Identify the population count columns and the percentage columns
count_columns = [col for col in df.columns if "_count" in col]
percentage_columns = [col for col in df.columns if col not in ['state', 'year'] and col not in count_columns]

# Convert percentage columns to decimal percentages
for col in percentage_columns:
    if '>' in col or '<' in col:  
        df[col] = df[col] / 100.0  
    else:
        count_col_name = col.split("_")[0] + "_count"
        if count_col_name in df.columns:  
            df[col] = df[col] / df[count_col_name]

# Display the updated DataFrame
print(df)
#NOPE

              state  year  18-24_count  18-24<hs_grad  18-24_hs_grad/equiv  \
0          Alabama   2016     463070.0         633.69             0.321129   
1           Alaska   2016      75647.0         105.53             0.392851   
2          Arizona   2016     673240.0        1045.86             0.329964   
3         Arkansas   2016     290440.0         451.97             0.331349   
4       California   2016    3852527.0        4505.16             0.298926   
5         Colorado   2016     534527.0         677.85             0.320710   
6      Connecticut   2016     356553.0         336.01             0.295235   
7         Delaware   2016      85281.0         143.46             0.346795   
8          Florida   2016    1763628.0        2728.53             0.305927   
9          Georgia   2016    1014366.0        1645.69             0.327040   
10          Hawaii   2016     129855.0          94.38             0.412668   
11           Idaho   2016     159418.0         191.33           

In [37]:
import pandas as pd


# Wrangles all non-percentage CSVs to convert the data to match 2010-2014
# Function that explicitly handles all columns for conversion based on pop. totals for specific demo.
def process_data(year):
    # Read the CSV file containing the data
    df = pd.read_csv(f"clean_ea_{year}.csv")

    # Convert numeric data to appropriate data types (not state or year cols)
    numeric_columns = df.columns[2:] 
    df[numeric_columns] = df[numeric_columns].replace(',', '', regex=True).astype(float)

    # Calculate percentages for the specified columns
    df['18-24<hs_grad'] = df['18-24<hs_grad'] / df['18-24_count']
    df['18-24_hs_grad/equiv'] = df['18-24_hs_grad/equiv'] / df['18-24_count']
    df['18-24_some_college/associate'] = df['18-24_some_college/associate'] / df['18-24_count']
    df['18-24_bachelor/higher'] = df['18-24_bachelor/higher'] / df['18-24_count']
    
    df['25+<9th_grade'] = df['25+<9th_grade'] / df['25+_count']
    df['25+_9th-12th_grade_nongrad'] = df['25+_9th-12th_grade_nongrad'] / df['25+_count']
    df['25+_hs_grad/equiv'] = df['25+_hs_grad/equiv'] / df['25+_count']
    df['25+_some_college'] = df['25+_some_college'] / df['25+_count']
    df['25+_associate\'s'] = df['25+_associate\'s'] / df['25+_count']
    df['25+_bachelor\'s'] = df['25+_bachelor\'s'] / df['25+_count']
    df['25+>bachelor'] = df['25+>bachelor'] / df['25+_count']
    
    df['25-34_hs_grad/higher'] = df['25-34_hs_grad/higher'] / df['25-34_count']
    df['25-34_bachelor/higher'] = df['25-34_bachelor/higher'] / df['25-34_count']
    
    df['35-44_hs_grad/higher'] = df['35-44_hs_grad/higher'] / df['35-44_count']
    df['35-44_bachelor/higher'] = df['35-44_bachelor/higher'] / df['35-44_count']
    
    df['45-64_hs_grad/higher'] = df['45-64_hs_grad/higher'] / df['45-64_count']
    df['45-64_bachelor/higher'] = df['45-64_bachelor/higher'] / df['45-64_count']
    
    df['65+_hs_grad/higher'] = df['65+_hs_grad/higher'] / df['65+_count']
    df['65+_bachelor/higher'] = df['65+_bachelor/higher'] / df['65+_count']
    

    # Save the formatted DataFrame as CSV by year
    df.to_csv(f"formatted_ea_{year}.csv", index=False)

# Run on CSVs for each year of data targeted
for year in range(2015, 2023):
    if year != 2020: 
        process_data(year)


In [72]:
def process_data(year):
    # Read the CSV file containing the data
    df = pd.read_csv(f"clean_ea_{year}.csv")

    # Drop the column "25+_earnings" and all columns that come after it
    df = df.loc[:, :"25+_earnings"]

    # Find the index of the column "25+_earnings"
    index_to_drop = df.columns.get_loc("25+_earnings")

    # Drop the "25+_earnings" column and all columns that come after it
    df = df.iloc[:, :index_to_drop]

    # Convert numeric data to appropriate data types (not state or year cols)
    numeric_columns = df.columns[2:]
    df[numeric_columns] = df[numeric_columns].replace(',', '', regex=True).astype(float)

    # Calculate percentages for the specified columns and round to 3 decimal places
    columns_18_24_to_process = ['18-24<hs_grad', '18-24_hs_grad/equiv', '18-24_some_college/associate', '18-24_bachelor/higher']
    columns_25_plus_to_process = ['25+<9th_grade', '25+_9th-12th_grade_nongrad', '25+_hs_grad/equiv', '25+_some_college', "25+_associate's", "25+_bachelor's", '25+>bachelor']
    columns_25_34_to_process = ['25-34_hs_grad/higher', '25-34_bachelor/higher']
    columns_35_44_to_process = ['35-44_hs_grad/higher', '35-44_bachelor/higher']
    columns_45_64_to_process = ['45-64_hs_grad/higher', '45-64_bachelor/higher']
    columns_65_plus_to_process = ['65+_hs_grad/higher', '65+_bachelor/higher']
    
    for column in columns_18_24_to_process:
        df[column] = round((df[column] / df['18-24_count']), 3) 
    
    for column in columns_25_plus_to_process:
        df[column] = round((df[column] / df['25+_count']), 3) 
        
    for column in columns_25_34_to_process:
        df[column] = round((df[column] / df['25-34_count']), 3)
        
    for column in columns_35_44_to_process:
        df[column] = round((df[column] / df['35-44_count']), 3)
        
    for column in columns_45_64_to_process:
        df[column] = round((df[column] / df['45-64_count']), 3)
        
    for column in columns_65_plus_to_process:
        df[column] = round((df[column] / df['65+_count']), 3)

    # Return the processed DataFrame
    return df

# Create an empty DataFrame to store all processed data
all_processed_data = pd.DataFrame()

# Run on CSVs for each year of data targeted
for year in range(2015, 2023):
    if year != 2020: 
        processed_df = process_data(year)
        # Concatenate the processed DataFrame with the existing DataFrame
        all_processed_data = pd.concat([all_processed_data, processed_df], ignore_index=True)

# Print the combined DataFrame
print("Combined Processed Data:")
all_processed_data.head()

Combined Processed Data:


,state,year,18-24_count,18-24<hs_grad,18-24_hs_grad/equiv,18-24_some_college/associate,18-24_bachelor/higher,25+_count,25+<9th_grade,25+_9th-12th_grade_nongrad,...,25-34_bachelor/higher,35-44_count,35-44_hs_grad/higher,35-44_bachelor/higher,45-64_count,45-64_hs_grad/higher,45-64_bachelor/higher,65+_count,65+_hs_grad/higher,65+_bachelor/higher
0,Alabama,2015,468319.0,0.152,0.319,0.454,0.075,3282252.0,0.048,0.103,...,0.258,612193.0,0.863,0.270,1283740.0,0.859,0.238,761847.0,0.803,0.212
1,Alaska,2015,82518.0,0.151,0.381,0.411,0.058,469523.0,0.026,0.048,...,0.283,91656.0,0.940,0.293,188081.0,0.936,0.306,71008.0,0.878,0.301
2,Arizona,2015,669583.0,0.171,0.304,0.452,0.074,4536954.0,0.060,0.079,...,0.264,845274.0,0.839,0.287,1656915.0,0.865,0.279,1120004.0,0.864,0.278
3,Arkansas,2015,284441.0,0.128,0.317,0.483,0.072,1987819.0,0.053,0.093,...,0.241,367499.0,0.879,0.251,756815.0,0.860,0.210,478114.0,0.800,0.186
4,California,2015,3943137.0,0.124,0.296,0.483,0.097,26085263.0,0.099,0.079,...,0.340,5190370.0,0.813,0.347,9872623.0,0.812,0.311,5189558.0,0.794,0.301


In [69]:
# List of file names
file_names = ["clean_ea_2010.csv", "clean_ea_2011.csv", "clean_ea_2012.csv", "clean_ea_2013.csv", "clean_ea_2014.csv"]

# Read each CSV file into a DataFrame and store them in a list
data_frames = [pd.read_csv(file) for file in file_names]

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Find the index of the column "25+_earnings"
index_to_drop = combined_df.columns.get_loc("25+_earnings")

# Drop the "25+_earnings" column and all columns that come after it
combined_df = combined_df.iloc[:, :index_to_drop]

# List of column names containing percentages
columns_to_convert = ['18-24<hs_grad', '18-24_hs_grad/equiv', '18-24_some_college/associate', '18-24_bachelor/higher', '25+<9th_grade', 
                      '25+_9th-12th_grade_nongrad', '25+_hs_grad/equiv', '25+_some_college', "25+_associate's", "25+_bachelor's", 
                      '25+>bachelor', '25-34_hs_grad/higher', '25-34_bachelor/higher', '35-44_hs_grad/higher', '35-44_bachelor/higher', 
                      '45-64_hs_grad/higher', '45-64_bachelor/higher', '65+_hs_grad/higher', '65+_bachelor/higher'] 

# Convert percentages to decimal format
combined_df[columns_to_convert] = combined_df[columns_to_convert].replace('%', '', regex=True).astype(float) / 100


In [70]:
combined_df.head()

,state,year,18-24_count,18-24<hs_grad,18-24_hs_grad/equiv,18-24_some_college/associate,18-24_bachelor/higher,25+_count,25+<9th_grade,25+_9th-12th_grade_nongrad,...,25-34_bachelor/higher,35-44_count,35-44_hs_grad/higher,35-44_bachelor/higher,45-64_count,45-64_hs_grad/higher,45-64_bachelor/higher,65+_count,65+_hs_grad/higher,65+_bachelor/higher
0,Alabama,2010,"488,349",0.199,0.301,0.439,0.060,"3,161,521",0.062,0.117,...,0.248,"621,094",0.854,0.231,"1,280,800",0.851,0.226,"657,974",0.723,0.167
1,Alaska,2010,"75,023",0.188,0.371,0.407,0.034,"450,577",0.028,0.061,...,0.245,"93,460",0.922,0.307,"200,088",0.923,0.294,"53,339",0.829,0.244
2,Arizona,2010,"634,568",0.196,0.307,0.436,0.060,"4,146,758",0.063,0.081,...,0.237,"823,770",0.850,0.268,"1,576,753",0.873,0.274,"886,795",0.837,0.245
3,Arkansas,2010,"291,054",0.203,0.325,0.415,0.056,"1,919,422",0.063,0.107,...,0.225,"371,843",0.862,0.210,"759,457",0.854,0.196,"420,075",0.725,0.152
4,California,2010,"3,944,934",0.173,0.276,0.473,0.078,"24,097,200",0.105,0.088,...,0.307,"5,191,065",0.797,0.316,"9,313,187",0.819,0.306,"4,269,495",0.766,0.265


In [100]:
api_df = pd.read_csv("API_Poverty.csv")

# Reorder the columns
api_df = api_df[['state', 'year', 'median_h_income', 'poverty_rate', 'poverty_count']]

#Assuming api_df is your DataFrame
api_df = api_df[api_df['state'] != 'District of Columbia']

# Reset index if needed
api_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the DataFrame
api_df.tail()

,state,year,median_h_income,poverty_rate,poverty_count
550,Alabama,2022,59703,0.162,798469
551,Alaska,2022,88072,0.108,77305
552,Arizona,2022,74355,0.125,904567
553,Arkansas,2022,55505,0.163,481650
554,California,2022,91517,0.122,4673718
555,Colorado,2022,89096,0.095,542333
556,Connecticut,2022,88182,0.098,345710
557,Delaware,2022,81933,0.100,99843
558,Florida,2022,69287,0.127,2770789
559,Georgia,2022,72742,0.129,1372488


In [80]:
# Assuming all_processed_data and combined_df are already defined

# Get the first few rows of all_processed_data and combined_df
years_df = pd.concat([combined_df, all_processed_data], ignore_index=True)

# Display the combined DataFrame
years_df.tail()


,state,year,18-24_count,18-24<hs_grad,18-24_hs_grad/equiv,18-24_some_college/associate,18-24_bachelor/higher,25+_count,25+<9th_grade,25+_9th-12th_grade_nongrad,...,25-34_bachelor/higher,35-44_count,35-44_hs_grad/higher,35-44_bachelor/higher,45-64_count,45-64_hs_grad/higher,45-64_bachelor/higher,65+_count,65+_hs_grad/higher,65+_bachelor/higher
595,Virginia,2022,842750.0,0.095,0.362,0.394,0.149,5978782.0,0.035,0.049,...,0.451,1180820.0,0.927,0.478,2179298.0,0.915,0.418,1462042.0,0.880,0.359
596,Washington,2022,671550.0,0.127,0.364,0.351,0.158,5470209.0,0.035,0.043,...,0.432,1115638.0,0.917,0.436,1869132.0,0.912,0.375,1309142.0,0.928,0.355
597,West Virginia,2022,161047.0,0.109,0.419,0.366,0.107,1262566.0,0.034,0.074,...,0.294,216666.0,0.917,0.310,462646.0,0.885,0.232,376162.0,0.860,0.207
598,Wisconsin,2022,562259.0,0.106,0.372,0.377,0.146,4088500.0,0.024,0.041,...,0.386,742993.0,0.939,0.392,1509392.0,0.932,0.319,1102386.0,0.927,0.274
599,Wyoming,2022,51982.0,0.161,0.327,0.441,0.072,399796.0,0.021,0.041,...,0.292,81607.0,0.918,0.323,137767.0,0.946,0.305,107988.0,0.934,0.265


In [110]:
# Merge the DataFrames using a full outer join on "state" and "year"
years_df = years_df.drop(['state', 'year'], axis=1)

# Concatenate the DataFrames along columns
all_clean_df = pd.concat([api_df, years_df], axis=1)

# Display the combined DataFrame
all_clean_df

,state,year,median_h_income,poverty_rate,poverty_count,18-24_count,18-24<hs_grad,18-24_hs_grad/equiv,18-24_some_college/associate,18-24_bachelor/higher,...,25-34_bachelor/higher,35-44_count,35-44_hs_grad/higher,35-44_bachelor/higher,45-64_count,45-64_hs_grad/higher,45-64_bachelor/higher,65+_count,65+_hs_grad/higher,65+_bachelor/higher
0,Alabama,2010,40538,0.189,883078,"488,349",0.199,0.301,0.439,0.060,...,0.248,"621,094",0.854,0.231,"1,280,800",0.851,0.226,"657,974",0.723,0.167
1,Alaska,2010,63456,0.110,76850,"75,023",0.188,0.371,0.407,0.034,...,0.245,"93,460",0.922,0.307,"200,088",0.923,0.294,"53,339",0.829,0.244
2,Arizona,2010,46787,0.176,1105075,"634,568",0.196,0.307,0.436,0.060,...,0.237,"823,770",0.850,0.268,"1,576,753",0.873,0.274,"886,795",0.837,0.245
3,Arkansas,2010,38413,0.187,529710,"291,054",0.203,0.325,0.415,0.056,...,0.225,"371,843",0.862,0.210,"759,457",0.854,0.196,"420,075",0.725,0.152
4,California,2010,57664,0.158,5785036,"3,944,934",0.173,0.276,0.473,0.078,...,0.307,"5,191,065",0.797,0.316,"9,313,187",0.819,0.306,"4,269,495",0.766,0.265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Virginia,2022,85838,0.106,893559,842750.0,0.095,0.362,0.394,0.149,...,0.451,1180820.0,0.927,0.478,2179298.0,0.915,0.418,1462042.0,0.880,0.359
596,Washington,2022,91255,0.100,764405,671550.0,0.127,0.364,0.351,0.158,...,0.432,1115638.0,0.917,0.436,1869132.0,0.912,0.375,1309142.0,0.928,0.355
597,West Virginia,2022,54097,0.174,299275,161047.0,0.109,0.419,0.366,0.107,...,0.294,216666.0,0.917,0.310,462646.0,0.885,0.232,376162.0,0.860,0.207
598,Wisconsin,2022,71050,0.108,621380,562259.0,0.106,0.372,0.377,0.146,...,0.386,742993.0,0.939,0.392,1509392.0,0.932,0.319,1102386.0,0.927,0.274


In [111]:
all_clean_df.to_csv("resources/all_clean_data.csv", index=False)